# Fantasy Football Topic Modeling using LDA
By Daniel Jimenez

In [23]:
import pandas as pd
import pprint

In [24]:
def first_pass(text):
    text = str(text)
    #remove newline character code
    text = text.replace('\n',' ')
    #remove the ampersand character code
    text = text.replace('&amp',' ')
    #character code for a Zero-width space
    text = text.replace('&#x200B;',' ')
    #non-breaking space
    text = text.replace('nbsp',' ')

    return text

In [25]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
stopwords_english = stopwords.words('english')

import string

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

def second_pass(text):
    #make string lowercase
    text = str(text)
    text = text.lower()

    #remove https links
    text = re.sub(r"[(+*)]\S*https?:\S*[(+*)]", "", text)
    #remove https links with no brackets
    text = re.sub(r"http\S+", "", text)
    #remove link markups []
    #note that this will also remove comment fields with ["Delete"] 
    text = re.sub(r"[\(\[].*?[\)\]]", " ", text)

    # remove numbers
    text = re.sub(r'\d+', '', text)

    # remove puncuation
    text = re.sub(r'[^\w\s]', '', text)

    #tokenize
    #create bag of words
    tokens = nltk.word_tokenize(text)

    clean_text = []

    #remove stopwords, puncuation, and words of length 3 or smaller
    for token in tokens:
        if (token not in stopwords_english and token not in string.punctuation and len(token) > 3):
                clean_text.append(token)

    return clean_text

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dannyjimenez/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dannyjimenez/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dannyjimenez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV

import os
import re, nltk, spacy

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        exc_list = ['week', 'season', 'year', 'game', 'fantasy', 'league', 'weekend', 'sunday'] # words to exclude
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] and token.lemma_ not in exc_list 
                                   else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [27]:
# create document-term matrix
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum occurences of a word 
                             ngram_range=(1,2)                 # extract unigrams, bigrams, and trigrams
                            )

In [28]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=10,           # Number of topics
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      n_jobs = -1,               # Use all available CPUs
                                     )

In [29]:
def gridsearch(data_vectorized):
    # Define Search Param
    search_params = {'n_components': [10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}

    # Init the Model
    lda = LatentDirichletAllocation()

    # Init Grid Search Class
    model = GridSearchCV(lda, param_grid=search_params)

    # Do the Grid Search
    model.fit(data_vectorized)

    # Best Model
    best_lda_model = model.best_estimator_

    # Model Parameters
    print("Best Model's Params: ", model.best_params_)

    # Log Likelihood Score
    print("Best Log Likelihood Score: ", model.best_score_)

    # Perplexity
    print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))
    
    return best_lda_model

In [30]:
import pyLDAvis
import pyLDAvis.lda_model

def get_lda_vis(best_lda_model):
    pyLDAvis.enable_notebook()
    panel = pyLDAvis.lda_model.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
    return panel

## <font color='green'>Overperformers</font>

### <font color='green'>Quarterbacks</font>

In [31]:
df = pd.read_excel('player_reddit_raw.xlsx', sheet_name='QB Overperformers')
df['clean_text'] = df['body'].apply(first_pass)
df['clean_text'] = df['clean_text'].apply(second_pass)

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(df.clean_text, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

data_vectorized = vectorizer.fit_transform(data_lemmatized)
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

# Grid Search
best_lda_model = gridsearch(data_vectorized)

# Topic Model Visualization
get_lda_vis(best_lda_model)

LatentDirichletAllocation(learning_method='online', n_jobs=-1, random_state=100)
Best Model's Params:  {'learning_decay': 0.7, 'n_components': 10}
Best Log Likelihood Score:  -60829.57739711219
Model Perplexity:  355.52489785482993


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
1     -21.546801  21.132530       1        1  51.265491
2      22.884430 -63.467022       2        1  10.854080
6     -44.801502 -15.546502       3        1  10.486487
8      59.666527 -33.111797       4        1   5.631255
9      24.311611 -27.381548       5        1   5.521895
7      -9.483023 -18.855856       6        1   4.516321
4      18.989662  41.115837       7        1   4.125039
5     -17.477966 -54.515354       8        1   3.524265
0      14.692924   6.126385       9        1   2.258448
3      52.184814   8.231620      10        1   1.816719, topic_info=        Term        Freq       Total Category  logprob  loglift
838   target  690.000000  690.000000  Default  30.0000  30.0000
962     yard  558.000000  558.000000  Default  29.0000  29.0000
336     help  369.000000  369.000000  Default  28.0000  28.0000
540     pass  507.000000  507.000000  Default  27.0000  27.0000
843     team  432.000000  432.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
463     mean    7.922866   86.917792  Topic10  -4.6407   1.6129
38    answer    7.903980  106.834945  Topic10  -4.6431   1.4042
454     make    8.650872  257.250558  Topic10  -4.5528   0.6157
433     link    7.842267  104.587201  Topic10  -4.6510   1.4176
135  comment    7.007117   60.411095  Topic10  -4.7636   1.8539

[556 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
3         1  0.101030          accord
3         3  0.808240          accord
3         4  0.033677          accord
3         8  0.033677          accord
4         3  0.964915  accord scoring
...     ...       ...             ...
962       6  0.189632            yard
962       8  0.012523            yard
967       1  0.970559     yard target
967       6  0.017331     yard target
971       1  0.989105            zone

[1375 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 7, 9, 10, 8, 5, 6, 1, 4])

### <font color='green'>Running Backs</font>

In [32]:
df = pd.read_excel('player_reddit_raw.xlsx', sheet_name='RB Overperformers')
df['clean_text'] = df['body'].apply(first_pass)
df['clean_text'] = df['clean_text'].apply(second_pass)

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(df.clean_text, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

data_vectorized = vectorizer.fit_transform(data_lemmatized)
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

# Grid Search
best_lda_model = gridsearch(data_vectorized)

# Topic Model Visualization
get_lda_vis(best_lda_model)

LatentDirichletAllocation(learning_method='online', n_jobs=-1, random_state=100)
Best Model's Params:  {'learning_decay': 0.7, 'n_components': 10}
Best Log Likelihood Score:  -48576.06819107059
Model Perplexity:  359.95339747679344


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
6     -114.014565   63.488232       1        1  47.004122
1       79.223343  -19.624031       2        1  12.967064
0       14.978059   22.476856       3        1  10.497414
9      -61.738235 -102.647621       4        1   6.888301
8      -16.497658  -42.980068       5        1   4.881923
2       37.088562  -95.244118       6        1   3.745967
5     -114.606422  -34.403728       7        1   3.583451
4       63.367653   77.976639       8        1   3.570939
7      -57.177864   16.924187       9        1   3.488316
3      -27.322134   95.221809      10        1   3.372503, topic_info=       Term        Freq       Total Category  logprob  loglift
625  target  759.000000  759.000000  Default  30.0000  30.0000
445   point  597.000000  597.000000  Default  29.0000  29.0000
418    pass  436.000000  436.000000  Default  28.0000  28.0000
720    yard  452.000000  452.000000  Default  27.0000  27.0000
533    rush  199.000000  199.000000  Default  26.0000  26.0000
..      ...         ...         ...      ...      ...      ...
222     get    9.869181  166.894942  Topic10  -4.7817   0.5616
39     back   10.402352  356.241965  Topic10  -4.7291  -0.1441
481    rank    9.183515  160.254063  Topic10  -4.8537   0.5302
605   still    9.425699  231.375945  Topic10  -4.8277   0.1889
592   start    9.291395  326.824409  Topic10  -4.8420  -0.1708

[546 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
2         1  0.070878          accord
2         2  0.850542          accord
2         7  0.035439          accord
3         2  0.946180  accord scoring
4         1  0.360277          achane
...     ...       ...             ...
721       9  0.282081      yard carry
721      10  0.056416      yard carry
723       1  0.220528       yard line
723       9  0.735095       yard line
724       1  0.980425     yard target

[1586 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 2, 1, 10, 9, 3, 6, 5, 8, 4])

### <font color='green'>Wide Receivers</font>

In [33]:
df = pd.read_excel('player_reddit_raw.xlsx', sheet_name='WR Overperformers')
df['clean_text'] = df['body'].apply(first_pass)
df['clean_text'] = df['clean_text'].apply(second_pass)

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(df.clean_text, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

data_vectorized = vectorizer.fit_transform(data_lemmatized)
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

# Grid Search
best_lda_model = gridsearch(data_vectorized)

# Topic Model Visualization
get_lda_vis(best_lda_model)

LatentDirichletAllocation(learning_method='online', n_jobs=-1, random_state=100)
Best Model's Params:  {'learning_decay': 0.9, 'n_components': 10}
Best Log Likelihood Score:  -67357.13338450507
Model Perplexity:  447.2494215878746


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
9     -15.573908  49.957596       1        1  49.759882
3      39.444378 -32.144028       2        1  16.017426
5      46.923885  10.306571       3        1   6.621288
6      11.935790 -10.585642       4        1   6.594106
8      -6.254532 -41.861332       5        1   6.131569
4      30.913208  51.876205       6        1   3.967915
7      10.316216  23.761271       7        1   3.522999
2     -41.793400 -21.945488       8        1   2.961728
1     -18.589470   5.099494       9        1   2.948732
0     -48.208496  24.531147      10        1   1.474356, topic_info=        Term        Freq       Total Category  logprob  loglift
815   target  994.000000  994.000000  Default  30.0000  30.0000
935     yard  621.000000  621.000000  Default  29.0000  29.0000
546     pass  534.000000  534.000000  Default  28.0000  28.0000
381   injury  309.000000  309.000000  Default  27.0000  27.0000
700     rush  184.000000  184.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
177  defense    8.857759  425.460277  Topic10  -4.3693   0.3451
30      also    6.066459  169.845723  Topic10  -4.7478   0.8848
446     look    7.537006  316.572589  Topic10  -4.5307   0.4792
546     pass    7.773157  534.425785  Topic10  -4.4999  -0.0136
580    point    5.718144  796.817954  Topic10  -4.8069  -0.7200

[565 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         1  0.375579  ability
0         4  0.046947  ability
0         5  0.046947  ability
0         6  0.469474  ability
2         1  0.393442  absence
...     ...       ...      ...
944       1  0.323521     zach
944       2  0.035947     zach
944       4  0.107840     zach
944       7  0.035947     zach
944       9  0.467309     zach

[1645 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 4, 6, 7, 9, 5, 8, 3, 2, 1])

### <font color='green'>Tight Ends</font>

In [34]:
df = pd.read_excel('player_reddit_raw.xlsx', sheet_name='TE Overperformers')
df['clean_text'] = df['body'].apply(first_pass)
df['clean_text'] = df['clean_text'].apply(second_pass)

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(df.clean_text, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

data_vectorized = vectorizer.fit_transform(data_lemmatized)
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

# Grid Search
best_lda_model = gridsearch(data_vectorized)

# Topic Model Visualization
get_lda_vis(best_lda_model)

LatentDirichletAllocation(learning_method='online', n_jobs=-1, random_state=100)
Best Model's Params:  {'learning_decay': 0.7, 'n_components': 10}
Best Log Likelihood Score:  -43674.50216580134
Model Perplexity:  337.0819168226845


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
1      30.858364 -75.552864       1        1  64.405964
6     -31.076954 -70.636848       2        1   9.270306
9      -8.580744  77.349373       3        1   8.062287
7     -65.769302 -18.896833       4        1   4.766011
4     -38.078640  29.871168       5        1   4.487167
0      61.183651 -28.433605       6        1   2.896131
3      -1.771982 -21.279734       7        1   1.878495
8      24.525364  22.850248       8        1   1.667541
5      53.292950  72.568565       9        1   1.599210
2      87.931755  20.914076      10        1   0.966888, topic_info=       Term        Freq       Total Category  logprob  loglift
577  target  882.000000  882.000000  Default  30.0000  30.0000
389    pass  512.000000  512.000000  Default  29.0000  29.0000
413   point  671.000000  671.000000  Default  28.0000  28.0000
587    team  357.000000  357.000000  Default  27.0000  27.0000
486  rookie  233.000000  233.000000  Default  26.0000  26.0000
..      ...         ...         ...      ...      ...      ...
89     come    2.875990  118.797431  Topic10  -4.6992   0.9178
403    play    4.239799  448.735106  Topic10  -4.3110  -0.0231
560   still    3.142537  223.314285  Topic10  -4.6105   0.3753
446    rank    2.924909  205.518590  Topic10  -4.6823   0.3866
535    snap    2.732020  137.687238  Topic10  -4.7505   0.7189

[521 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1         2  0.753261     accord
1         3  0.188315     accord
2         1  0.624215     achane
2         8  0.109238     achane
2         9  0.249686     achane
...     ...       ...        ...
674       7  0.081454       zach
674       8  0.040727       zach
674      10  0.407271       zach
675       4  0.097664  zach ertz
675      10  0.781311  zach ertz

[1321 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 7, 10, 8, 5, 1, 4, 9, 6, 3])

## <font color='red'>Underperformers</font>

### <font color='red'>Quarterbacks</font>

In [35]:
df = pd.read_excel('player_reddit_raw.xlsx', sheet_name='QB Underperformers')
df['clean_text'] = df['body'].apply(first_pass)
df['clean_text'] = df['clean_text'].apply(second_pass)

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(df.clean_text, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

data_vectorized = vectorizer.fit_transform(data_lemmatized)
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

# Grid Search
best_lda_model = gridsearch(data_vectorized)

# Topic Model Visualization
get_lda_vis(best_lda_model)

LatentDirichletAllocation(learning_method='online', n_jobs=-1, random_state=100)
Best Model's Params:  {'learning_decay': 0.9, 'n_components': 10}
Best Log Likelihood Score:  -80684.61812367346
Model Perplexity:  374.12769567065044


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
9     -28.220396  34.851967       1        1  40.789994
7      17.190670 -19.960127       2        1  14.559569
1      13.109301  47.590160       3        1  12.446552
8      38.242172 -49.229877       4        1  10.100665
2      -4.760914   7.534260       5        1   9.159328
3      30.049179  12.840343       6        1   3.771278
4      59.955204 -11.883843       7        1   3.302105
5      -9.523872 -45.144512       8        1   2.489933
6     -35.411449 -12.699097       9        1   1.958058
0      58.266869  34.623962      10        1   1.422517, topic_info=        Term        Freq       Total Category  logprob  loglift
1009  target  908.000000  908.000000  Default  30.0000  30.0000
1158    yard  610.000000  610.000000  Default  29.0000  29.0000
402     help  664.000000  664.000000  Default  28.0000  28.0000
695    point  872.000000  872.000000  Default  27.0000  27.0000
1019    team  492.000000  492.000000  Default  26.0000  26.0000
...      ...         ...         ...      ...      ...      ...
1166   young    8.125111   77.355553  Topic10  -4.6438   1.9993
675     play   16.167104  619.054739  Topic10  -3.9558   0.6075
521     list    8.528148  174.830519  Topic10  -4.5954   1.2323
766   raider    6.654380   70.229054  Topic10  -4.8435   1.8963
590    night    6.524262   69.788890  Topic10  -4.8633   1.8828

[594 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
5         4  0.945553  accord scoring
6         1  0.343451         account
6         6  0.601040         account
7         2  0.379205        accuracy
7         4  0.037920        accuracy
...     ...       ...             ...
1167      1  0.365149            zach
1167      2  0.199172            zach
1167      4  0.099586            zach
1167      9  0.265563            zach
1167     10  0.033195            zach

[1534 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 8, 2, 9, 3, 4, 5, 6, 7, 1])

### <font color='red'>Running Backs</font>

In [36]:
df = pd.read_excel('player_reddit_raw.xlsx', sheet_name='RB Underperformers')
df['clean_text'] = df['body'].apply(first_pass)
df['clean_text'] = df['clean_text'].apply(second_pass)

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(df.clean_text, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

data_vectorized = vectorizer.fit_transform(data_lemmatized)
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

# Grid Search
best_lda_model = gridsearch(data_vectorized)

# Topic Model Visualization
get_lda_vis(best_lda_model)

LatentDirichletAllocation(learning_method='online', n_jobs=-1, random_state=100)
Best Model's Params:  {'learning_decay': 0.9, 'n_components': 10}
Best Log Likelihood Score:  -54468.317458885234
Model Perplexity:  384.7920583006863


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
5     -56.940006 -63.899624       1        1  45.814070
6      80.322754 -45.704529       2        1   8.348499
0      76.873260  26.841312       3        1   8.040419
7     -45.837025  70.991882       4        1   7.469127
4      32.376564  92.553307       5        1   7.011121
3      10.959934 -88.742714       6        1   6.789272
1     -36.529148  -3.620155       7        1   5.647091
9      20.309868 -25.959366       8        1   5.466846
2      10.745373  34.463848       9        1   2.753479
8     -95.899323  12.904706      10        1   2.660077, topic_info=          Term        Freq       Total Category  logprob  loglift
690     target  900.000000  900.000000  Default  30.0000  30.0000
804       yard  522.000000  522.000000  Default  29.0000  29.0000
457       pass  456.000000  456.000000  Default  28.0000  28.0000
554  reception  175.000000  175.000000  Default  27.0000  27.0000
594       rush  227.000000  227.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
594       rush    7.790716  227.237076  Topic10  -4.8832   0.2538
204     expect    7.733724  224.204003  Topic10  -4.8906   0.2598
480     player    8.205987  368.133133  Topic10  -4.8313  -0.1768
616        see    6.748004   93.915468  Topic10  -5.0269   0.9937
30        also    6.957348  176.392024  Topic10  -4.9964   0.3939

[585 rows x 6 columns], token_table=      Topic      Freq                Term
term                                     
0         3  0.930614          abbreviate
1         3  0.930614  abbreviate version
4         1  0.074593            absolute
4         7  0.820522            absolute
5         1  0.256594          absolutely
...     ...       ...                 ...
809       1  0.986080         yard target
812       1  0.700864                zone
812       4  0.027485                zone
812       5  0.178652                zone
812      10  0.082455                zone

[1738 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 7, 1, 8, 5, 4, 2, 10, 3, 9])

### <font color='red'>Wide Receivers</font>

In [37]:
df = pd.read_excel('player_reddit_raw.xlsx', sheet_name='WR Underperformers')
df['clean_text'] = df['body'].apply(first_pass)
df['clean_text'] = df['clean_text'].apply(second_pass)

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(df.clean_text, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

data_vectorized = vectorizer.fit_transform(data_lemmatized)
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

# Grid Search
best_lda_model = gridsearch(data_vectorized)

# Topic Model Visualization
get_lda_vis(best_lda_model)

LatentDirichletAllocation(learning_method='online', n_jobs=-1, random_state=100)
Best Model's Params:  {'learning_decay': 0.5, 'n_components': 10}
Best Log Likelihood Score:  -35661.13346031407
Model Perplexity:  166.54170432231493


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
2     -113.388870  -74.031303       1        1  39.801642
0       23.581818   12.291338       2        1  27.463859
1      -50.464275   42.592262       3        1   9.540556
7      -82.489975  117.271873       4        1   8.788073
5       18.718246  104.603996       5        1   6.443851
8      -12.442968 -114.893860       6        1   6.066969
6       66.356453  -61.136257       7        1   1.366326
9       99.136108   41.911659       8        1   0.294176
4      -39.562672  -36.704948       9        1   0.173308
3     -132.051193   19.692373      10        1   0.061240, topic_info=       Term        Freq       Total Category  logprob  loglift
439   start  363.000000  363.000000  Default  30.0000  30.0000
160    help  714.000000  714.000000  Default  29.0000  29.0000
294    play  321.000000  321.000000  Default  28.0000  28.0000
467    team  286.000000  286.000000  Default  27.0000  27.0000
475  thread  484.000000  484.000000  Default  26.0000  26.0000
..      ...         ...         ...      ...      ...      ...
465  target    0.033385  309.392173  Topic10  -6.3641  -1.7361
16     also    0.033385   84.650207  Topic10  -6.3641  -0.4401
169    high    0.033385   71.117333  Topic10  -6.3642  -0.2659
294    play    0.033384  321.523949  Topic10  -6.3642  -1.7746
149    good    0.033384  206.261053  Topic10  -6.3642  -1.3307

[508 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
1         2  0.097454          accord
1         4  0.877086          accord
2         4  0.971685  accord scoring
4         3  0.537840   accurate time
4         6  0.440051   accurate time
...     ...       ...             ...
527       5  0.154822            yard
527       7  0.025804            yard
528       2  0.884778            zone
528       5  0.075838            zone
528       7  0.025279            zone

[953 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2, 8, 6, 9, 7, 10, 5, 4])

### <font color='red'>Tight Ends</font>

In [38]:
df = pd.read_excel('player_reddit_raw.xlsx', sheet_name='TE Underperformers')
df['clean_text'] = df['body'].apply(first_pass)
df['clean_text'] = df['clean_text'].apply(second_pass)

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(df.clean_text, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

data_vectorized = vectorizer.fit_transform(data_lemmatized)
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

# Grid Search
best_lda_model = gridsearch(data_vectorized)

# Topic Model Visualization
get_lda_vis(best_lda_model)

LatentDirichletAllocation(learning_method='online', n_jobs=-1, random_state=100)
Best Model's Params:  {'learning_decay': 0.9, 'n_components': 10}
Best Log Likelihood Score:  -30915.57040678447
Model Perplexity:  257.28316142639414


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
8     -74.621605  16.579287       1        1  34.302506
5     -69.396362 -36.222511       2        1  16.504659
0     -17.502712 -68.533325       3        1  15.022414
4     -24.754576 -22.877003       4        1   8.075205
6      57.175018  11.625114       5        1   6.825597
9     -38.990383  66.142563       6        1   5.396478
1      30.167330 -42.861317       7        1   4.861395
3      15.753412  52.839588       8        1   3.631545
2     -27.679453  21.864210       9        1   2.732562
7      12.624946   2.104191      10        1   2.647640, topic_info=              Term        Freq       Total Category  logprob  loglift
423         target  718.000000  718.000000  Default  30.0000  30.0000
428  target target  137.000000  137.000000  Default  29.0000  29.0000
338      reception  149.000000  149.000000  Default  28.0000  28.0000
366           rush  170.000000  170.000000  Default  27.0000  27.0000
280           pass  382.000000  382.000000  Default  26.0000  26.0000
..             ...         ...         ...      ...      ...      ...
409          still    6.480951  199.704196  Topic10  -4.5807   0.2035
293           play    7.124678  370.632788  Topic10  -4.4860  -0.3201
132         finish    5.611091  134.290606  Topic10  -4.7248   0.4562
429           team    5.814079  367.807407  Topic10  -4.6893  -0.5158
400          start    5.640906  287.434015  Topic10  -4.7195  -0.2994

[548 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
1         1  0.070053          accord
1         3  0.840634          accord
1         7  0.070053          accord
2         3  0.971170  accord scoring
3         1  0.367617          achane
...     ...       ...             ...
496       1  0.145001            zach
496       2  0.507505            zach
496       3  0.072501            zach
496       5  0.145001            zach
496       8  0.072501            zach

[1714 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 6, 1, 5, 7, 10, 2, 4, 3, 8])

## Code Sources

The code in this notebook contains snippets from the following sources:
- [Tawfiq Ammari](https://github.com/tawfiqam/MI564/blob/main/LDA_Intro.ipynb)
- [Selva Prabhakaran](https://www.machinelearningplus.com/nlp/topic-modeling-python-sklearn-examples/)